# Computing seasonal means from CFSR data

## Introduction

These notes will show how to:
1. Access the 6-hourly CFSR dataset on our local disks
2. Use `xarray.open_mfdataset()` to concatenate data spread across many individual netCDF files into a single dataset
3. Take advantage of xarray's "lazy execution" model to define our climatology before explicitly computing any results
4. Compute the climatology (seasonal means) and save the resulting (vastly) reduced datasets to disk as netCDF files

***Note, this code takes a long time to run! And there is really no need to run it again because we are saving the results to disk.***

This is just for demonstration purposes.

## The CFSR data

The [Climate Forecase System Reanalysis](https://climatedataguide.ucar.edu/climate-data/climate-forecast-system-reanalysis-cfsr) or CFSR is a reanalysis data product giving a best estimate of the state of the coupled climate system since 1979 based on a blend of observations and numerical model.

> The CFSR is a third generation reanalysis product. It is a global, high resolution, coupled atmosphere-ocean-land surface-sea ice system designed to provide the best estimate of the state of these coupled domains over this period. The CFSR includes (1) coupling of atmosphere and ocean during the generation of the 6 hour guess field, (2) an interactive sea-ice model, and (3) assimilation of satellite radiances. The CFSR global atmosphere resolution is ~38 km (T382) with 64 levels. The global ocean is 0.25° at the equator, extending to a global 0.5° beyond the tropics, with 40 levels. The global land surface model has 4 soil levels and the global sea ice model has 3 levels. The CFSR atmospheric model contains observed variations in carbon dioxide (CO2), together with changes in aerosols and other trace gases and solar variations. With these variable parameters, the analyzed state will include estimates of changes in the Earth system climate due to these factors. The current CFSR will be extended as an operational, real time product into the future.

We maintain a continuously updated local copy of the 6-hourly CFSR data on shared disks here in DAES. You can browse the available data through this catalog:

http://thredds.atmos.albany.edu:8080/thredds/catalog/CFSR/catalog.html

That links takes you to a so-called THREDDS server which provides remote access to the data. However we're always going to get *much* better performance with local access to the file system.

In [2]:
thredds_path = 'http://thredds.atmos.albany.edu:8080/thredds/dodsC/CFSR/' 
local_path = '/cfsr/data/'

path = local_path  # switch this to thredds_path if running remotely ... but much poorer performance

Let's take a peak at one directory of data from a particular year:

In [4]:
ls /cfsr/data/2000/

g.2000.0p5.anl.nc        q.2000.0p5.anl.nc       u_pv.2000.0p5.anl.nc
pmsl.2000.0p5.anl.nc     t.2000.0p5.anl.nc       v.2000.0p5.anl.nc
pres_pv.2000.0p5.anl.nc  t_pv.2000.0p5.anl.nc    v_isen.2000.0p5.anl.nc
psfc.2000.0p5.anl.nc     tsfc.2000.0p5.anl.nc    v_pv.2000.0p5.anl.nc
pv_isen.2000.0p5.anl.nc  u.2000.0p5.anl.nc       w.2000.0p5.anl.nc
pwat.2000.0p5.anl.nc     u_isen.2000.0p5.anl.nc


We can see 17 different fields stored in individual netCDF files labeled with their calendar year.

Our goal is to **compute a 30-year seasonal climatology** for all variables. So we are going to begin by opening handles to 30 years of data.

In [14]:
# It's helpful to store all the variable names in this list
variables = ['g',
             'pmsl',
             'pres_pv',
             'psfc',
             'pv_isen',
             'pwat',
             'q',
             't',
             't_pv',
             'tsfc',
             'u',
             'u_isen',
             'u_pv',
             'v',
             'v_isen',
             'v_pv',
             'w',
            ]

## Opening handles to the dataset

Here we are going to assemble a list of paths to every data file that we want to concatenate into a single dataset.

### Assembling paths to all the data

In [21]:
files = []
for y in range(1988,2018):  # choosing years 1988 through 2017 because there is some missing data after 2017
    for var in variables:
        filepath = path + str(y) + '/' + var + '.' + str(y) +'.0p5.anl.nc'
        files.append(filepath)

Let's take a peek at part of the list we just created:

In [22]:
files[:24]

['/cfsr/data/1988/g.1988.0p5.anl.nc',
 '/cfsr/data/1988/pmsl.1988.0p5.anl.nc',
 '/cfsr/data/1988/pres_pv.1988.0p5.anl.nc',
 '/cfsr/data/1988/psfc.1988.0p5.anl.nc',
 '/cfsr/data/1988/pv_isen.1988.0p5.anl.nc',
 '/cfsr/data/1988/pwat.1988.0p5.anl.nc',
 '/cfsr/data/1988/q.1988.0p5.anl.nc',
 '/cfsr/data/1988/t.1988.0p5.anl.nc',
 '/cfsr/data/1988/t_pv.1988.0p5.anl.nc',
 '/cfsr/data/1988/tsfc.1988.0p5.anl.nc',
 '/cfsr/data/1988/u.1988.0p5.anl.nc',
 '/cfsr/data/1988/u_isen.1988.0p5.anl.nc',
 '/cfsr/data/1988/u_pv.1988.0p5.anl.nc',
 '/cfsr/data/1988/v.1988.0p5.anl.nc',
 '/cfsr/data/1988/v_isen.1988.0p5.anl.nc',
 '/cfsr/data/1988/v_pv.1988.0p5.anl.nc',
 '/cfsr/data/1988/w.1988.0p5.anl.nc',
 '/cfsr/data/1989/g.1989.0p5.anl.nc',
 '/cfsr/data/1989/pmsl.1989.0p5.anl.nc',
 '/cfsr/data/1989/pres_pv.1989.0p5.anl.nc',
 '/cfsr/data/1989/psfc.1989.0p5.anl.nc',
 '/cfsr/data/1989/pv_isen.1989.0p5.anl.nc',
 '/cfsr/data/1989/pwat.1989.0p5.anl.nc',
 '/cfsr/data/1989/q.1989.0p5.anl.nc']

and the last few items too:

In [23]:
files[-24:]

['/cfsr/data/2016/u.2016.0p5.anl.nc',
 '/cfsr/data/2016/u_isen.2016.0p5.anl.nc',
 '/cfsr/data/2016/u_pv.2016.0p5.anl.nc',
 '/cfsr/data/2016/v.2016.0p5.anl.nc',
 '/cfsr/data/2016/v_isen.2016.0p5.anl.nc',
 '/cfsr/data/2016/v_pv.2016.0p5.anl.nc',
 '/cfsr/data/2016/w.2016.0p5.anl.nc',
 '/cfsr/data/2017/g.2017.0p5.anl.nc',
 '/cfsr/data/2017/pmsl.2017.0p5.anl.nc',
 '/cfsr/data/2017/pres_pv.2017.0p5.anl.nc',
 '/cfsr/data/2017/psfc.2017.0p5.anl.nc',
 '/cfsr/data/2017/pv_isen.2017.0p5.anl.nc',
 '/cfsr/data/2017/pwat.2017.0p5.anl.nc',
 '/cfsr/data/2017/q.2017.0p5.anl.nc',
 '/cfsr/data/2017/t.2017.0p5.anl.nc',
 '/cfsr/data/2017/t_pv.2017.0p5.anl.nc',
 '/cfsr/data/2017/tsfc.2017.0p5.anl.nc',
 '/cfsr/data/2017/u.2017.0p5.anl.nc',
 '/cfsr/data/2017/u_isen.2017.0p5.anl.nc',
 '/cfsr/data/2017/u_pv.2017.0p5.anl.nc',
 '/cfsr/data/2017/v.2017.0p5.anl.nc',
 '/cfsr/data/2017/v_isen.2017.0p5.anl.nc',
 '/cfsr/data/2017/v_pv.2017.0p5.anl.nc',
 '/cfsr/data/2017/w.2017.0p5.anl.nc']

Looks good!

### Passing through paths to `xarray.open_mfdataset()`

Now we pass our list of path and let Xarray (with Dask in the background) handle the concatenation:

In [12]:
import xarray as xr

cfsr_6hourly = xr.open_mfdataset(files, parallel=True)
cfsr_6hourly

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/k

<xarray.Dataset>
Dimensions:  (lev: 40, time: 43832, lat: 361, lon: 720)
Coordinates:
  * lev      (lev) float64 -2e-06 2e-06 10.0 20.0 ... 925.0 950.0 975.0 1e+03
  * time     (time) datetime64[ns] 1988-01-01 ... 2017-12-31T18:00:00
  * lat      (lat) float32 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float32 -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Data variables: (12/17)
    g        (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    pmsl     (time, lat, lon) float32 dask.array<chunksize=(1464, 361, 720), meta=np.ndarray>
    pres_pv  (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    psfc     (time, lat, lon) float32 dask.array<chunksize=(1464, 361, 720), meta=np.ndarray>
    pv_isen  (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    pwat     (time, lat, lon) float32 dask.array<chunksize=(1464, 361, 720), meta=np.ndarray>
    ...       ...
    u_isen   (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    u_pv     (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    v        (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    v_isen   (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    v_pv     (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    w        (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
Attributes:
    description:    g 1000-10 hPa
    year:           1988
    source:         http://nomads.ncdc.noaa.gov/data.php?name=access#CFSR-data
    references:     Saha, et. al., (2010)
    created_by:     User: kgriffin
    creation_date:  Fri Apr 13 14:12:27 UTC 2012

In [26]:
cfsr_6hourly.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:  (lev: 40, time: 43832, lat: 361, lon: 720)
Coordinates:
  * lev      (lev) float64 -2e-06 2e-06 10.0 20.0 ... 925.0 950.0 975.0 1e+03
  * time     (time) datetime64[ns] 1988-01-01 ... 2017-12-31T18:00:00
  * lat      (lat) float32 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float32 -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Data variables: (12/17)
    g        (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    pmsl     (time, lat, lon) float32 dask.array<chunksize=(1464, 361, 720), meta=np.ndarray>
    pres_pv  (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    psfc     (time, lat, lon) float32 dask.array<chunksize=(1464, 361, 720), meta=np.ndarray>
    pv_isen  (time, lev, lat, lon) float32 dask.array<chunksize=(1464, 40, 361, 720), meta=np.ndarray>
    pwat     (time, lat, lon) float32 dask.array<chunksiz

Ignoring for now the long list of warning, we see that we have a dataset with
- 17 different data variables
- 1/4 degree lat-lon resolution
- 40 pressure levels
- 43,832 time elements (that's 4x daily for 30 years, accounting for leap years!)

### Dask and lazy execution

Note that we've just loaded in handles to a LOT of data. Each 4-dimensional field (lat,lon,pressure,time) is roughly $4.6 \times 10^{11}$ data points, which at single-precision floating point storage works out to about 1.7 TB!

The total size of this 30-year dataset is about 22 TB.

How does this work? We definitely do not have 22 TB of available memory on this system!

This is an example of "lazy execution". We have not actually read all that data off of disk. All we have done is read the metadata to effectively create a map of the dataset so that Xarray knows which computations should be valid on this dataset.

## Creating the seasonal climatology

Now let's get started on *reducing* this dataset into a form more amenable to our analysis. We are going to take *time averages* over individual seasons.

### Grouping by season

Our time access already understands meteorological seasons DJF (December, January, February), JJA (June, July, August), etc. So creating groups for the four seasons is very simple. Here's a quick example of using a `groupby` operation:

In [9]:
cfsr_6hourly.groupby(cfsr_6hourly.time.dt.season)

DatasetGroupBy, grouped over 'season'
4 groups with labels 'DJF', 'JJA', 'MAM', 'SON'.

### Definining the climatology

We use a grouped operation to take averages over all data points within each season.

*Note, this operation is very fast because no actualy computations are being done at this point, just laying out a map for **how to do the computations**.*

In [10]:
cfsr_seas_clim = cfsr_6hourly.groupby(cfsr_6hourly.time.dt.season).mean()
cfsr_seas_clim

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/k

<xarray.Dataset>
Dimensions:  (lev: 40, lat: 361, lon: 720, season: 4)
Coordinates:
  * lev      (lev) float64 -2e-06 2e-06 10.0 20.0 ... 925.0 950.0 975.0 1e+03
  * lat      (lat) float32 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float32 -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables: (12/17)
    g        (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    pmsl     (season, lat, lon) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    pres_pv  (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    psfc     (season, lat, lon) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    pv_isen  (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    pwat     (season, lat, lon) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    ...       ...
    u_isen   (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    u_pv     (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    v        (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    v_isen   (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    v_pv     (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>
    w        (season, lev, lat, lon) float32 dask.array<chunksize=(1, 40, 361, 720), meta=np.ndarray>

Now we can see that the resulting dataset has a `seasons` axis with our four seasons 'DJF', 'MAM', 'JJA', 'SON'.

*With this time averaging, we have reduced the data from 22 TB down to about 2 GB.*

Our task is now to save the reduced dataset to disk, so that we can do our analyses on data that fits much more comfortably into memory.

*Now comes all the heavy lifting.* In order to save the climatology to disk, we have to actually carry out the computations. This part will be very slow.

## Saving the climatology to disk

The code is extremely simple to write, but the execution will be very slow.

In [ ]:
cfsr_seas_clim.to_netcdf('/nfs/roselab_rit/data/CFSR_seas_clim.nc')